In [ ]:
# Rong
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

train_path = 'gdrive/My Drive/MiniP-2/train.csv'
test_path = 'gdrive/My Drive/MiniP-2/test.csv'

Mounted at /content/gdrive


In [ ]:
import numpy as np
import pandas as pd
import sklearn as sl
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Normalizer
from sklearn import metrics

# Part 1. Using the data provided in train.csv

## 1. Implement Bernoulli Naive Bayes

##2. Additional experiments using other classifiers

In [ ]:
train = pd.read_csv(train_path)
train_X = train['body']
train_Y = train['subreddit']

#test = pd.read_csv(test_path)
#test_X = test['body']
#train.head()



### 1. Logistic Regression

In [ ]:
tf_idf_vectorizer = TfidfVectorizer(stop_words='english')

x_train, x_test, y_train, y_test = train_test_split(train_X, train_Y, train_size=0.7, test_size=0.3)

vectors_train = tf_idf_vectorizer.fit_transform(x_train)
vectors_test = tf_idf_vectorizer.transform(x_test)

normalizer_train = Normalizer().fit(X=vectors_train)

vectors_train_normalized = normalizer_train.transform(vectors_train)
vector_test_normalized = normalizer_train.transform(vectors_test)

lr_model = LogisticRegression()


lr_model.fit(vectors_train, y_train)
lr_pred = lr_model.predict(vector_test_normalized)

print((y_test == lr_pred).mean())
print(metrics.classification_report(y_test, lr_pred))


###2. LinearSVC

In [ ]:
tf_idf_vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(1,2))

x_train, x_test, y_train, y_test = train_test_split(train_X, train_Y, train_size=0.7, test_size=0.3)

vectors_train = tf_idf_vectorizer.fit_transform(x_train)
vectors_test = tf_idf_vectorizer.transform(x_test)

normalizer_train = Normalizer().fit(X=vectors_train)

vectors_train_normalized = normalizer_train.transform(vectors_train)
vector_test_normalized = normalizer_train.transform(vectors_test)

l_SVC_model = LinearSVC(class_weight='balanced',multi_class='crammer_singer')

l_SVC_model.fit(vectors_train, y_train)
l_SVC_pred = l_SVC_model.predict(vector_test_normalized)

print(metrics.accuracy_score(y_test, l_SVC_pred))
print(metrics.classification_report(y_test, l_SVC_pred))


In [ ]:
from sklearn.model_selection import GridSearchCV

n_sample = len(x_train)
tuned_parameters = [{'C': [1, 10, 100, 1000, 10000],}]
scores = ['precision', 'recall']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(
        LinearSVC(), tuned_parameters, scoring='%s_macro' % score
    )
    clf.fit(vectors_train,  y_train)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, clf.predict(X_test)
    print(classification_report(y_true, y_pred))
    print()

###3. Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

tf_idf_vectorizer = TfidfVectorizer(stop_words='english')

x_train, x_test, y_train, y_test = train_test_split(train_X, train_Y, train_size=0.7, test_size=0.3)

vectors_train = tf_idf_vectorizer.fit_transform(x_train)
vectors_test = tf_idf_vectorizer.transform(x_test)

normalizer_train = Normalizer().fit(X=vectors_train)

vectors_train_normalized = normalizer_train.transform(vectors_train)
vector_test_normalized = normalizer_train.transform(vectors_test)

dt_model = DecisionTreeClassifier()

dt_model.fit (vectors_train, y_train)
dt_pred = dt_model.predict(vector_test_normalized)

print(metrics.accuracy_score(y_test, dt_pred))
print(metrics.classification_report(y_test, dt_pred))



0.6886330935251799
              precision    recall  f1-score   support

       anime       0.78      0.79      0.78       372
        cars       0.78      0.79      0.79       645
   computers       0.43      0.32      0.36       130
 datascience       0.78      0.83      0.80       730
     gamedev       0.49      0.47      0.48       296
   gamernews       0.41      0.43      0.42       235
    hardware       0.62      0.62      0.62       489
         rpg       0.73      0.68      0.70       578

    accuracy                           0.69      3475
   macro avg       0.63      0.62      0.62      3475
weighted avg       0.68      0.69      0.69      3475



## 3. Model Validation

In [ ]:
from sklearn.model_selection import KFold
from copy import deepcopy

def model_valid(model, features_X, target_Y, vectorizer, k_fold=3):
    kf = KFold(n_splits=k_fold)
    counter = 1
    ave = 0
       
    for train_index, test_index in kf.split(features_X, target_Y):
      tempMod = deepcopy(model)
      vectors_train = vectorizer.fit_transform(features_X[train_index])
      vectors_test = vectorizer.transform(features_X[test_index])
      normalizer_train = Normalizer().fit(X=vectors_train)
      vectors_train_normalized = normalizer_train.transform(vectors_train)
      vectors_test_normalized = normalizer_train.transform(vectors_test)
      
      tempMod.fit(vectors_train, target_Y[train_index])
      #y_pred = tempMod.predict(vector_test_normalized)
      # ave += (y_pred == target_Y[test_index]).mean() 
      # print(ave)
      # print(counter)
      # counter+=1

    return #ave/k_fold;


# Classifier Running Default


## Logistic Regression default

In [ ]:
from sklearn.linear_model import LogisticRegression

tf_idf_vectorizer = TfidfVectorizer()

vectors_X = tf_idf_vectorizer.fit_transform(train_X)

vectors_train, vectors_test, y_train, y_test = train_test_split(vectors_X, train_Y, train_size=0.7, test_size=0.3)


normalizer_train = Normalizer().fit(X=vectors_train)
vectors_train_normalized = normalizer_train.transform(vectors_train)
vector_test_normalized = normalizer_train.transform(vectors_test)


lr_model = LogisticRegression()

lr_model.fit(vectors_train, y_train)
lr_pred = lr_model.predict(vector_test_normalized)

print(metrics.accuracy_score(y_test, lr_pred))
print(metrics.classification_report(y_test, lr_pred))




0.8627338129496402
              precision    recall  f1-score   support

       anime       0.95      0.90      0.92       404
        cars       0.85      0.97      0.90       603
   computers       0.86      0.37      0.52       118
 datascience       0.89      0.95      0.92       736
     gamedev       0.90      0.66      0.76       300
   gamernews       0.85      0.55      0.66       222
    hardware       0.79      0.86      0.82       495
         rpg       0.84      0.94      0.89       597

    accuracy                           0.86      3475
   macro avg       0.87      0.77      0.80      3475
weighted avg       0.87      0.86      0.86      3475



/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


## Decision Tree Defalut

In [ ]:
from sklearn.tree import DecisionTreeClassifier

dt_model = DecisionTreeClassifier()

dt_model.fit (vectors_train, y_train)
dt_pred = dt_model.predict(vector_test_normalized)

print(metrics.accuracy_score(y_test, dt_pred))
print(metrics.classification_report(y_test, dt_pred))

0.695251798561151
              precision    recall  f1-score   support

       anime       0.81      0.76      0.79       389
        cars       0.79      0.79      0.79       629
   computers       0.43      0.35      0.39       138
 datascience       0.82      0.82      0.82       744
     gamedev       0.52      0.48      0.50       326
   gamernews       0.41      0.50      0.45       228
    hardware       0.65      0.62      0.64       460
         rpg       0.67      0.71      0.69       561

    accuracy                           0.70      3475
   macro avg       0.64      0.63      0.63      3475
weighted avg       0.70      0.70      0.70      3475



## Linear support vector machine default

In [ ]:
from sklearn.svm import LinearSVC 

l_SVC_model = LinearSVC()

l_SVC_model.fit(vectors_train, y_train)
l_SVC_pred = l_SVC_model.predict(vector_test_normalized)

print(metrics.accuracy_score(y_test, l_SVC_pred))
print(metrics.classification_report(y_test, l_SVC_pred))




0.8776978417266187
              precision    recall  f1-score   support

       anime       0.96      0.95      0.95       389
        cars       0.91      0.94      0.92       629
   computers       0.84      0.63      0.72       138
 datascience       0.92      0.96      0.94       744
     gamedev       0.84      0.76      0.80       326
   gamernews       0.70      0.67      0.68       228
    hardware       0.83      0.81      0.82       460
         rpg       0.88      0.91      0.90       561

    accuracy                           0.88      3475
   macro avg       0.86      0.83      0.84      3475
weighted avg       0.88      0.88      0.88      3475



In [ ]:
train = pd.read_csv(train_path)
#test = pd.read_csv(test_path)
#test_X = test['body']
#train.head()

train_X = train['body']
train_Y = train['subreddit']


# X_train, X_test, y_train, y_test = train_test_split(train_X, train_Y, train_size=0.7, test_size=0.3)


tf_idf_vectorizer = TfidfVectorizer(binary=True, max_features=5000, stop_words='english')

vectors_X = tf_idf_vectorizer.fit_transform(train_X)

vectors_train, vectors_test, y_train, y_test = train_test_split(vectors_X, train_Y, train_size=0.7, test_size=0.3)

# vectors_train = tf_idf_vectorizer.fit_transform(X_train)

#print(vectors_train)

# vectors_test = tf_idf_vectorizer.fit_transform(X_test)


normalizer_train = Normalizer().fit(X=vectors_train)
vectors_train_normalized = normalizer_train.transform(vectors_train)
vector_test_normalized = normalizer_train.transform(vectors_test)

#print(vectors_train_normalized)


BNB = BernoulliNB()

BNB.fit(vectors_train_normalized, y_train)

y_pred = BNB.predict(vector_test_normalized)


print (metrics.accuracy_score(y_test, y_pred)) 

print(metrics.classification_report(y_test, y_pred))
